# importing our libraries

In [1]:
! pip install tensorflow_text 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.8 MB/s eta 0:00:00


In [2]:
import tensorflow as tf  #It is the machine learning package used to build the neural network.

import tensorflow_hub as hub  #  It contains a pre-trained machine model used to build our text classification. Our pre-trained model is BERT. 
#We will re-use the BERT model and fine-tune it to meet our needs.

import tensorflow_text as text  #It will allow us to work with text

import pandas as pd

# let load and explore our dataset

In [3]:
df = pd.read_csv('/content/text_data.csv')

In [4]:
df.head()

,Unnamed: 0,utts,labels
0,142,well â€¡ the stool is falling over . the boy i...,0
1,355,these two little kids are getting cookies out ...,1
2,79,mhm . there is a boy and a girl and the boy is...,0
3,366,the girls trying ... did I get that ? and gir...,1
4,215,there is a girl waiting to get a cookie from I...,0


In [5]:
text = df['utts'].values.tolist()

In [6]:
df['labels'].value_counts()

1    309
0    243
Name: labels, dtype: int64

# spletting the dataset

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['utts'], df['labels'])

# Getting started with BERT

BERT stands for Bidirectional Encoder Representations from Transformers. BERT models help machines understand and interpret the meaning of the text. It uses immediately preceding text to understand the context. It also checks the relationships of words within a sentence to give the actual meaning of words.

In [9]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3") # downloading preprocess bert model to preprocess our dataset
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4") # downloading the encoding BERT MODEL TO ENCODE OUR DATASET

In [10]:
# Initializing the BERT layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [11]:
# initializing the network neural layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [12]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [14]:
# model compiling
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [15]:
# fitting the model
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
13/13 [==============================] - 19s 331ms/step - loss: 0.7902 - accuracy: 0.5169 - precision: 0.5515 - recall: 0.7186
Epoch 2/10
13/13 [==============================] - 4s 317ms/step - loss: 0.7044 - accuracy: 0.5145 - precision: 0.5949 - recall: 0.4069
Epoch 3/10
13/13 [==============================] - 4s 314ms/step - loss: 0.7054 - accuracy: 0.5459 - precision: 0.5570 - recall: 0.9091
Epoch 4/10
13/13 [==============================] - 4s 340ms/step - loss: 0.7116 - accuracy: 0.5266 - precision: 0.6129 - recall: 0.4113
Epoch 5/10
13/13 [==============================] - 4s 316ms/step - loss: 0.6931 - accuracy: 0.5242 - precision: 0.5521 - recall: 0.7792
Epoch 6/10
13/13 [==============================] - 4s 318ms/step - loss: 0.6891 - accuracy: 0.5362 - precision: 0.5575 - recall: 0.8182
Epoch 7/10
13/13 [==============================] - 4s 345ms/step - loss: 0.6711 - accuracy: 0.5845 - precision: 0.6105 - recall: 0.7056
Epoch 8/10
13/13 [======================

# Evaluating the model using the test dataset

In [16]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

5/5 [==============================] - 2s 351ms/step


In [17]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 1])